In [1]:
import numpy as np
import os
import sys
import pandas as pd


import seaborn as sns

import pylab as pl
import h5py

import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIBLE_DEVICES']="0" 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3/'
os.chdir(base_dir)

from GlobalUtils import GlobalUtils
utils = GlobalUtils()
os.chdir(utils.base_dir)
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.RamLoader import RamLoader
gen_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3/Classes/DataProcessing'
os.chdir(gen_dir)
import ts_RamGenerator
os.chdir(base_dir)


helper = HelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules

import random
import pprint


1 Physical GPUs, 1 Logical GPUs


In [2]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'noise_not_noise' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 1,
    'seed' : 1,
    'subsample_size' : 0.1,
    'balance_non_train_set' : True,
    'use_true_test_set' : False,
    'even_balance' : True
}
loadData = LoadData(**load_args)
full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)

2 2
Mapping redundancy: [--------------------------------------->] 100 %



Loaded explosion and earthquake dataset:
Evenly balanced among classes in the train set.
As well as non train sets.
Distribution (Label: (counts, proportion)) of
Full ds:
earthquake: (10554, 0.5066)  |  explosion: (10279, 0.4934)  
Train ds:
earthquake: (7962, 0.5096)  |  explosion: (7662, 0.4904)  
Val ds:
earthquake: (1567, 0.5014)  |  explosion: (1558, 0.4986)  
Test ds:
earthquake: (1025, 0.4918)  |  explosion: (1059, 0.5082)  


In [28]:
from Classes.DataProcessing.LoadData import LoadData
import os
import sys
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

import os
base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3'
os.chdir(base_dir)

from GlobalUtils import GlobalUtils
utils = GlobalUtils()

class GridSearchResultProcessor():

    def __init__(self, num_classes):
        self.num_classes = num_classes
        
    
    def create_results_df(self, hyper_picks, model_picks):
        hyper_keys = list(hyper_picks.keys())
        model_keys = list(model_picks.keys())
        metrics_train_keys = ["train_loss", "train_accuracy", "train_precision", "train_recall"]
        metrics_val_keys = ["val_loss", "val_accuracy", "val_precision", "val_recall"]
        header = np.concatenate((hyper_keys, model_keys, metrics_train_keys, metrics_val_keys))
        results_df = pd.DataFrame(columns = header)
        return results_df

    def create_results_df_opti(self, current_picks):
        keys = list(current_picks.keys())
        metrics_train_keys = ["train_loss", "train_accuracy", "train_precision", "train_recall"]
        metrics_val_keys = ["val_loss", "val_accuracy", "val_precision", "val_recall"]
        confusion_matrix_key = ["confusion_matrix"]
        header = np.concatenate((keys, metrics_train_keys, metrics_val_keys, confusion_matrix_key))
        results_df = pd.DataFrame(columns = header)
        return results_df
    
    def initiate_results_df(self, file_name, num_classes, start_from_scratch, hyper_picks, model_picks):
        if start_from_scratch:
            self.clear_results_df(file_name)
            return self.create_results_df(hyper_picks, model_picks)
        else:
            if self.does_result_exist(file_name):
                file_name = file_name.split('/')[-1]
                results_df = self.get_results_df_by_name(file_name)
                return results_df
            else:
                return self.create_results_df(hyper_picks, model_picks)

    def initiate_results_df_opti(self, file_name, num_classes, start_from_scratch, search_picks):
        if start_from_scratch:
            self.clear_results_df(file_name)
            return self.create_results_df_opti(search_picks)
        else:
            if self.does_result_exist(file_name) or self.does_result_exist(f"{self.get_results_file_path()}/{file_name}"):
                #file_name = file_name.split('/')[-1]
                results_df = self.get_results_df_by_name(file_name)
                return results_df
            else:
                return self.create_results_df_opti(search_picks)

        
    def does_result_exist(self, file_name):
        if isfile(f"{self.get_results_file_path()}/{file_name}"):
            return True
        return isfile(file_name)
        
        
    """
    def save_results_df(self, results_df, file_name):
        results_df.to_csv(file_name, mode = 'w', index=False)
    """
    def clear_results_df(self, file_name):
        path = self.get_results_file_path()
        file = f"{path}/{file_name}"
        if os.path.isfile(file):
            f = open(file, "w+")
            f.close()
        
    
    def get_results_file_name(self, narrow = False, narrowOpt = False):
        file_name = f"results_{self.model_nr_type}"
        if narrow:
            file_name = f"{file_name}_NARROW"
        if narrowOpt:
            file_name = f"{file_name}_NarrowOpt"
        if self.loadData.earth_explo_only:
            file_name = f"{file_name}_earthExplo"
        if self.loadData.noise_earth_only:
            file_name = f"{file_name}_noiseEarth"
        if self.loadData.noise_not_noise:
            file_name = f"{file_name}_noiseNotNoise"
        if self.filter_name != None:
            file_name = f"{file_name}_{self.filter_name}"
            if self.filter_name == "bandpass":
                file_name = f"{file_name}-{self.band_min}-{self.band_max}"
            if self.filter_name == "highpass":
                file_name = f"{file_name}-{self.highpass_freq}"
        if self.use_time_augmentor:
            file_name = f"{file_name}_timeAug"
        if self.scaler_name:
            file_name == file_name + f"{self.scaler_name}"
        if self.use_noise_augmentor:
            file_name = f"{file_name}_noiseAug"
        if self.use_early_stopping:
            file_name = f"{file_name}_earlyS"
        file_name = file_name + f"_numChannels-{self.num_channels}"
        file_name = file_name + ".csv"
        return file_name
    
    def get_results_file_path(self):
        file_path = f'{utils.base_dir}/GridSearchResults/{self.num_classes}_classes'
        return file_path
    
    def store_params_before_fit(self, current_picks, results_df, file_name):

        hyper_params = current_picks[1]
        model_params = current_picks[2]
        picks = []
        for key in list(hyper_params.keys()):
            picks.append(hyper_params[key])
        for key in list(model_params.keys()):
            picks.append(model_params[key])
        nr_fillers = len(results_df.columns) - len(picks)
        for i in range(nr_fillers):
            picks.append(np.nan)
        temp_df = pd.DataFrame(np.array(picks, dtype = object).reshape(1,len(results_df.columns)), columns = results_df.columns)
        results_df = results_df.append(temp_df, ignore_index = True)
        for idx, column in enumerate(results_df.columns):
            if idx >= len(picks):
                results_df[column] = results_df[column].astype('float')
        self.save_results_df(results_df, file_name)
        return results_df

    def store_params_before_fit_opti(self, current_picks, results_df, file_name):
        columns = results_df.columns
        filled_dict = {}
        for column in columns:
            if column not in list(current_picks.keys()):
                current_picks[column] = np.nan
            filled_dict[column] = [current_picks[column]]
        for idx, column in enumerate(columns):
            assert column == list(filled_dict.keys())[idx], print(f"True order: {columns}. Created order: {list(filled_dict.keys())}")
        temp = pd.DataFrame.from_dict(filled_dict, orient = "columns")
        temp = temp.reindex(results_df.columns, axis = 1)
        results_df = results_df.append(temp, ignore_index = True)
        self.save_results_df(results_df, file_name)
        return results_df



    def store_metrics_after_fit(self, metrics, confusion_matrix, results_df, file_name):
        results_df = results_df.replace('nan', np.nan)
        unfinished_columns = results_df.columns[results_df.isna().any()].tolist()
        for column in unfinished_columns:
            if column == "confusion_matrix":
                results_df.iloc[-1, results_df.columns.get_loc(column)] = str(np.array(confusion_matrix))
            else:
                results_df.iloc[-1, results_df.columns.get_loc(column)] = metrics[column.split('_')[0]][column]
        self.save_results_df(results_df, file_name)
        return results_df

    
    def find_best_performers(self, results_df):
        train_loss_index = results_df.columns.get_loc('train_loss')
        metrics_df = results_df[results_df.columns[train_loss_index:]]
        min_loss = {'train_loss' : min(metrics_df['train_loss']), 'val_loss' : min(metrics_df['val_loss']), 
                    'train_index' : metrics_df[metrics_df['train_loss'] == min(metrics_df['train_loss'])].index[0], 
                    'val_index' : metrics_df[metrics_df['val_loss'] == min(metrics_df['val_loss'])].index[0]}

        max_accuracy = {'train_accuracy' : max(metrics_df['train_accuracy']), 'val_accuracy' : max(metrics_df['val_accuracy']), 
                        'train_index' : metrics_df[metrics_df['train_accuracy'] == max(metrics_df['train_accuracy'])].index[0], 
                        'val_index' : metrics_df[metrics_df['val_accuracy'] == max(metrics_df['val_accuracy'])].index[0]}

        max_precision = {'train_precision' : max(metrics_df['train_precision']), 'val_precision' : max(metrics_df['val_precision']), 
                         'train_index' : metrics_df[metrics_df['train_precision'] == max(metrics_df['train_precision'])].index[0], 
                         'val_index' : metrics_df[metrics_df['val_precision'] == max(metrics_df['val_precision'])].index[0]}

        max_recall = {'train_recall' : max(metrics_df['train_recall']), 'val_recall' : max(metrics_df['train_recall']), 
                      'train_index' : metrics_df[metrics_df['train_recall'] == max(metrics_df['train_recall'])].index[0], 
                      'val_index' : metrics_df[metrics_df['val_recall'] == max(metrics_df['val_recall'])].index[0]}

        return min_loss, max_accuracy, max_precision, max_recall
    
    """
    TODO: The functions below are largely redundant, and should be streamlined in the future

    They are included as is in order to speed up the development of NarrowOptimizer
    """

    def get_results_df_by_name(self, file_name):
        file_path = f"{utils.base_dir}/GridSearchResults/{self.num_classes}_classes"
        loaded_df = pd.read_csv(file_path+'/'+file_name)
        return loaded_df


    def clear_nans(self, result_file_name):
        df = self.get_results_df_by_name(result_file_name)
        df_copy = df.copy()
        no_nans = df_copy.dropna()
        self.clear_results_df(result_file_name)
        self.save_results_df(no_nans, result_file_name)

    def save_results_df(self, results_df, file_name):
        #results_df.to_csv(file_name, mode = 'w', index=False)
        print(f"Saving file. {len(results_df)} rows.")
        print(f"{file_name} saved to path:   {self.get_results_file_path()}")
        results_df.to_csv(f"{self.get_results_file_path()}/{file_name}", mode = 'w', index=False)






In [31]:
import numpy as np
import pandas as pd
import h5py
import gc

import sklearn as sk

from sklearn.model_selection import ParameterGrid




import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.utils import GeneratorEnqueuer

import os
base_dir = '/media/tord/T7/Thesis_ssd/MasterThesis3'
os.chdir(base_dir)

from Classes.Modeling.DynamicModels import DynamicModels
from Classes.Modeling.StaticModels import StaticModels
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.HelperFunctions import HelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.RamLoader import RamLoader
#from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.DataProcessing.ts_RamGenerator import data_generator


import sys


import random
import pprint
import re
import json

class RGS(GridSearchResultProcessor):
    
    def __init__(self, loadData, train_ds, val_ds, test_ds, model_type, scaler_name, use_time_augmentor, use_noise_augmentor,
                 filter_name, n_picks, hyper_grid, use_tensorboard = False, 
                 use_liveplots = True, use_custom_callback = False, use_early_stopping = False, use_reduced_lr = False,
                 band_min = 2.0, band_max = 4.0, highpass_freq = 0.1, start_from_scratch = True, is_lstm = False, 
                 log_data = True, num_channels = 3):
        
        self.loadData = loadData
        self.train_ds = train_ds
        self.val_ds = val_ds
        self.test_ds = test_ds
        self.model_nr_type = model_type
        self.num_classes = len(set(self.loadData.label_dict.values()))

        self.scaler_name = scaler_name
        self.use_noise_augmentor = use_noise_augmentor
        self.use_time_augmentor = use_time_augmentor
        self.filter_name = filter_name
        self.n_picks = n_picks
        self.hyper_grid = hyper_grid
        self.use_tensorboard = use_tensorboard
        self.use_liveplots = use_liveplots
        self.use_custom_callback = use_custom_callback
        self.use_reduced_lr = use_reduced_lr
        self.use_early_stopping = use_early_stopping

        self.band_min = band_min
        self.band_max = band_max
        self.highpass_freq = highpass_freq
        self.start_from_scratch = start_from_scratch
        self.is_lstm = is_lstm
        self.log_data = log_data
        self.num_channels = num_channels
        
        self.helper = HelperFunctions()
        self.handler = DataHandler(self.loadData)
        self.is_dynamic = False
        if type(self.model_nr_type) == str:
            self.is_dynamic = True

            

    def fit(self):
        pp = pprint.PrettyPrinter(indent=4)
        # Creating grid:
        self.p = ParameterGrid(self.hyper_grid)
        if len(self.p) < self.n_picks:
            self.n_picks = len(self.p)
            print(f"Picks higher than max. Reducing picks to {self.n_picks} picks")
        self.p = self.get_n_params_from_list(self.p, self.n_picks)
        
        # Create name of results file, get initiated results df, either brand new or continue old.
        self.results_file_name = self.get_results_file_name()
        print(self.results_file_name)
        if self.start_from_scratch:
            confirmation = input("Are you sure you want to erase the results file? Y/N \n").upper()
            if confirmation != "Y":
                print("Terminating process.")
                return
        self.results_df = self.initiate_results_df_opti(self.results_file_name, self.num_classes, self.start_from_scratch, self.p[0])
        print(self.results_df)
        # Preprocessing and loading all data to RAM:
        ramLoader = RamLoader(self.loadData, 
                              self.handler, 
                              use_time_augmentor = self.use_time_augmentor, 
                              use_noise_augmentor = self.use_noise_augmentor, 
                              scaler_name = self.scaler_name,
                              filter_name = self.filter_name, 
                              band_min = self.band_min,
                              band_max = self.band_max,
                              highpass_freq = self.highpass_freq, 
                              load_test_set = False)
        self.x_train, self.y_train, self.x_val, self.y_val, self.timeAug, self.scaler, self.noiseAug = ramLoader.load_to_ram()

        
        
        for i in range(len(self.p)):
            gc.collect()
            tf.keras.backend.clear_session()
            tf.config.optimizer.set_jit(True)
            mixed_precision.set_global_policy('mixed_float16')

            model_info = {"model_nr_type" : self.model_nr_type, "index" : i}
            print(f"Model nr {i + 1} of {len(self.p)}")           
            # Translate picks to a more readable format:
            num_channels = self.num_channels
            epoch = self.p[i]["epochs"]
            batch_size = self.p[i]["batch_size"]
            
            opt = self.helper.get_optimizer(self.p[i]["optimizer"], self.p[i]["learning_rate"])
    

            if "decay_sequence" in self.p[i]:
                if "num_filters" in self.p[i]:
                    units_or_num_filters = self.p[i]["num_filters"]
                else:
                    units_or_num_filters = self.p[i]["units"]
                num_layers = self.p[i]["num_layers"]
                self.p[i]["decay_sequence"] = self.helper.get_max_decay_sequence(num_layers, 
                                                                                units_or_num_filters, 
                                                                                self.p[i]["decay_sequence"], 
                                                                                self.num_classes)
   
            if "first_dense_units" in self.p[i]:
                if self.p[i]["first_dense_units"] < self.p[i]["second_dense_units"]:
                    self.p[i]["second_dense_units"] = self.p[i]["first_dense_units"]
            
            current_picks = [model_info, self.p[i]]
            pp.pprint(current_picks)
            # Store picked parameters:
            if self.log_data:
                self.results_df = self.store_params_before_fit_opti(self.p[i], self.results_df, self.results_file_name)

            _, _, timesteps = self.x_train.shape

            input_shape = (timesteps, self.num_channels)
            
            if self.is_dynamic:  
                model = DynamicModels(self.model_nr_type, self.num_classes, input_shape, **self.p[i]).model
            else:
                raise Exception("Static models are not handled by this class yet.")
            
            # Initializing generators:
            #gen = RamGenerator(self.loadData, self.handler, self.noiseAug)
            train_enq = GeneratorEnqueuer(data_generator(self.x_train, self.y_train, batch_size, self.loadData, self.handler, self.noiseAug, num_channels = num_channels, is_lstm  = self.is_lstm), use_multiprocessing = False)
            val_enq = GeneratorEnqueuer(data_generator(self.x_val, self.y_val,batch_size, self.loadData, self.handler, self.noiseAug, num_channels = num_channels, is_lstm  = self.is_lstm), use_multiprocessing = False)
            train_enq.start(workers = 16, max_queue_size = 15)
            val_enq.start(workers = 16, max_queue_size = 15)
            train_gen = train_enq.get()
            val_gen = train_enq.get()

            # Generate compiler args using picks
            model_compile_args = self.helper.generate_model_compile_args(opt, self.num_classes)
            # Compile model using generated args
            model.compile(**model_compile_args)
            
            print("Starting: ")
            pp.pprint(self.p[i])
            print("---------------------------------------------------------------------------------")

            
            # Generate fit args using picks.
            fit_args = self.helper.generate_fit_args(self.train_ds, self.val_ds, batch_size, 
                                                     epoch, val_gen, use_tensorboard = self.use_tensorboard, 
                                                     use_liveplots = self.use_liveplots, 
                                                     use_custom_callback = self.use_custom_callback,
                                                     use_early_stopping = self.use_early_stopping,
                                                     use_reduced_lr = self.use_reduced_lr)
            try:
                print(f"Utilizes {self.helper.get_steps_per_epoch(self.loadData.val, batch_size)*batch_size}/{len(self.loadData.val)} validation points")
                print(f"Utilizes {self.helper.get_steps_per_epoch(self.loadData.train, batch_size)*batch_size}/{len(self.loadData.train)} training points")
                
                
                # Fit the model using the generated args
                model.fit(train_gen, **fit_args)
                
                # Evaluate the fitted model on the validation set
                val_eval = model.evaluate(x=val_gen,
                                          steps=self.helper.get_steps_per_epoch(self.loadData.val, batch_size),
                                          return_dict = True)
                pp.pprint(val_eval)
                
                metrics = {}
                metrics['val'] = {  "val_loss" : val_eval["loss"],
                                    "val_accuracy" : val_eval["binary_accuracy"],
                                    "val_precision": val_eval["precision"],
                                    "val_recall" : val_eval["recall"]}
                
                # Evaluate the fitted model on the train set
                
                train_eval = model.evaluate(x=train_gen,
                                            steps=self.helper.get_steps_per_epoch(self.loadData.train, batch_size),
                                            return_dict = True)
                train_enq.stop()
                val_enq.stop()
                
                metrics['train'] = { "train_loss" : train_eval["loss"],
                                    "train_accuracy" : train_eval["binary_accuracy"],
                                    "train_precision": train_eval["precision"],
                                    "train_recall" : train_eval["recall"]}
                
                
                conf, _ = self.helper.evaluate_model(model, self.x_val, self.y_val, self.loadData.label_dict, num_channels = self.num_channels, plot = False, run_evaluate = False)
                train_enq.stop()
                val_enq.stop()
                gc.collect()
                
                tf.keras.backend.clear_session()
                tf.compat.v1.reset_default_graph()
                del model, train_gen, val_gen, train_enq, val_enq
                
                if self.log_data:
                    self.results_df = self.store_metrics_after_fit(metrics, conf, self.results_df, self.results_file_name)

            except Exception as e:
                print(str(e))
                print("Something went wrong while training the model (most likely)")

                continue


    def print_best_performers(self, min_loss, max_accuracy, max_precision, max_recall):
        print("----------------------------------------------------LOSS----------------------------------------------------------")
        print(f"Min val loss: {min_loss['val_loss']}, at index: {min_loss['val_index']}")
        print(f"Min training loss: {min_loss['train_loss']}, at index: {min_loss['train_index']}")
        print("----------------------------------------------------ACCURACY------------------------------------------------------")
        print(f"Highest val accuracy: {max_accuracy['val_accuracy']}, at index: {max_accuracy['val_index']}")
        print(f"Highest training accuracy: {max_accuracy['train_accuracy']}, at index: {max_accuracy['train_index']}")
        print("----------------------------------------------------PRECISION-----------------------------------------------------")
        print(f"Highest val precision: {max_precision['val_precision']}, at index: {max_precision['val_index']}")
        print(f"Highest training precision: {max_precision['train_precision']}, at index: {max_precision['train_index']}") 
        print("-----------------------------------------------------RECALL-------------------------------------------------------")
        print(f"Highest val recall: {max_recall['val_recall']}, at index: {max_recall['val_index']}")
        print(f"Highest training recall: {max_recall['train_recall']}, at index: {max_recall['train_index']}")
        print("------------------------------------------------------------------------------------------------------------------")
       

    
    def get_n_params_from_list(self, grid, n_picks):
        print(f"Length of grid: {len(grid)}")
        indexes = random.sample(range(0, len(grid)), n_picks)
        picks = [grid[idx] for idx in indexes]
        return picks

In [33]:
# IncepTime grid:
"""
hyper_grid = {    
    "batch_size" : [64, 128, 256],
    "epochs" : [50, 50, 50, 50, 50, 50, 50, 50, 50],
    "learning_rate" : [0.1, 0.01, 0.01, 0.001, 0.0001],
    "optimizer" : ["adam", "adam", "adam", "adam", "rmsprop", "sgd", "adam", "rmsprop", "sgd"],
    "use_residuals" : [True, True, False],
    "use_bottleneck" : [True, True, False],
    "num_modules" : np.concatenate((np.array([1]), np.arange(3, 9, 3))),
    "filter_size" : np.arange(30, 60, 2),
    "bottleneck_size" : np.arange(30, 50, 2),
    "num_filters" : np.arange(20, 40, 2),
    "residual_activation" : ["relu", "relu", "relu", "relu", "tanh"],
    "module_activation" : ["linear", "linear", "linear", "relu", "tanh"],
    "module_output_activation" : ["relu", "relu", "relu", "relu", "linear", "tanh"],
    "output_layer_activation": ["sigmoid"],
    "reg_residual": [True, False, False],
    "reg_module" : [True, False, False],
    "l1_r" : [0.1, 0.01, 0.01, 0.001, 0.0001, 0],
    "l2_r" : [0.1, 0.01, 0.01, 0.001, 0.0001, 0]
}
"""
# Dense grid:

hyper_grid = {
        "batch_size" : [64, 128, 256],
        "epochs" : [1],
        "learning_rate" : [0.1, 0.01, 0.01, 0.001, 0.001, 0.0001, 0.0001],
        "optimizer" : ["sgd", "sgd", "sgd", "sgd", "rmsprop", "adam", "rmsprop", "sgd"],
        "num_layers" : [1, 2, 3, 4, 5],
        "units" : np.arange(100, 300, 10),
        "use_layerwise_dropout_batchnorm" : [False, True],
        "decay_sequence" : [[1,2,4,4,2,1], [1,4,8,8,4,1], [1,1,1,1,1,1], [1, 2, 4, 6, 8, 10]],
        "dropout_rate" : [0.3, 0.2, 0.1, 0.01, 0.001, 0],
        "l2_r" : [0.3, 0.2, 0.1, 0.01, 0.001, 0.0001],
        "l1_r" : [0.3, 0.2, 0.1, 0.01, 0.001, 0.0001],
        "activation" : ["tanh", "tanh", "relu", "relu", "relu", "sigmoid", "softmax"],
        "output_layer_activation" : ["sigmoid"]
    }



model_type = "DENSE"
is_lstm = True
num_channels = 3    

use_time_augmentor = True
scaler_name = "standard"
use_noise_augmentor = True
filter_name = None
band_min = 2.0
band_max = 4.0
highpass_freq = 15

n_picks = 100

use_tensorboard = False
use_liveplots = False
use_custom_callback = True
use_early_stopping = True
start_from_scratch = True
use_reduced_lr = True
log_data = True

shutdown = False

def clear_tensorboard_dir():
        import os
        import shutil
        path = f"{base_dir}/Tensorboard_dir/fit"
        files = os.listdir(path)
        print(files)
        for f in files:
            shutil.rmtree(os.path.join(path,f))
if use_tensorboard:
    clear_tensorboard_dir()



randomGridSearch = RGS(loadData, train_ds, val_ds, test_ds, model_type, scaler_name, use_time_augmentor, use_noise_augmentor,
                                            filter_name, n_picks, hyper_grid=hyper_grid, use_tensorboard = use_tensorboard, 
                                            use_liveplots = use_liveplots, use_custom_callback = use_custom_callback, use_early_stopping = use_early_stopping, 
                                            use_reduced_lr = use_reduced_lr, band_min = band_min, band_max = band_max, highpass_freq = highpass_freq, 
                                            start_from_scratch = start_from_scratch, is_lstm = is_lstm, log_data = log_data, num_channels = num_channels)
results_df, min_loss, max_accuracy, max_precision, max_recall = randomGridSearch.fit()


Length of grid: 203212800
results_DENSE_earthExplo_timeAug_noiseAug_earlyS_numChannels-3.csv
Are you sure you want to erase the results file? Y/N 
y
Empty DataFrame
Columns: [use_layerwise_dropout_batchnorm, units, output_layer_activation, optimizer, num_layers, learning_rate, l2_r, l1_r, epochs, dropout_rate, decay_sequence, batch_size, activation, train_loss, train_accuracy, train_precision, train_recall, val_loss, val_accuracy, val_precision, val_recall, confusion_matrix]
Index: []

[0 rows x 22 columns]
Fit process completed after 14.012139081954956 seconds. Total datapoints fitted: 31480.
Average time per datapoint: 0.0004451124231878957


Stage one loading training set, timeAug: [--------------------------------------->] 100 %

Stage one loading validation set, timeAug: [--------------------------------------->] 100 %

Fitting scaler progress: [------------------->] 100 %

Stage two loading training set, labels and standard scaler: [--------------------------------------->] 100 %

/home/tord/miniconda3/envs/thesis/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'l1_r': 0.3,
        'l2_r': 0.2,
        'learning_rate': 0.01,
        'num_layers': 1,
        'optimizer': 'sgd',
        'output_layer_activation': 'sigmoid',
        'units': 210,
        'use_layerwise_dropout_batchnorm': False}]
Saving file. 2 rows.
results_DENSE_earthExplo_timeAug_noiseAug_earlyS_numChannels-3.csv saved to path:   /media/tord/T7/Thesis_ssd/MasterThesis3/GridSearchResults/2_classes
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 6000, 3)]         0         
_________________________________________________________________
flatten (Flatten)            (None, 18000)             0         
_________________________________________________________________
dense (Dense)                (None, 210)               3780210   
_________________________________________________________________
dense_1 (Dense)              (None, 1)         

KeyboardInterrupt: 

In [ ]:
input_layer = tf.keras.layers.Input((1))
x = input_layer
x = tf.keras.layers.Dense(1)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs = input_layer, outputs = output)

In [ ]:
model.compile()

In [ ]:
model.fit(x = [[1],[0]], y = [1,0])

In [20]:
a = np.array([[1,2],[4,5]])
a

array([[1, 2],
       [4, 5]])

In [25]:
b = pd.DataFrame([[1,1,np.nan],[2,2,np.nan]], columns = ["a","b","c"])

In [26]:
b

,a,b,c
0,1,1,NaN
1,2,2,NaN


In [27]:
b.loc[-2, b.columns == "c"] = list(a)

ValueError: Must have equal len keys and value when setting with an ndarray

In [24]:
b

,a,b,c
0,1.0,1.0,NaN
1,2.0,2.0,NaN
-2,NaN,NaN,[[1 2]\n [4 5]]
